In [1]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.1 MB/s eta 0:00:00


In [2]:
# Install tensorflow
!pip install tensorflow

In [3]:
import tensorflow as tf
import pickle
import numpy as np
import string
from pyvi import ViTokenizer
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
# --- BƯỚC 2: LOAD DỮ LIỆU ---
# Giả sử bạn đã upload file lên Colab hoặc Mount Drive
# Thay đổi đường dẫn path phù hợp
path_sequences = '/content/sequences_digit.pkl' # [cite: 10]
path_tokenizer = '/content/tokenizer.pkl'       # [cite: 15]

# Load sequences
with open(path_sequences, 'rb') as f:
    sequences_digit = pickle.load(f) # [cite: 14]

# Load tokenizer
with open(path_tokenizer, 'rb') as f:
    loaded_tokenizer = pickle.load(f) # [cite: 19]

# Thiết lập thông số từ điển
vocab_size = len(loaded_tokenizer.word_index) + 1 # [cite: 20]
print(f"Vocab size: {vocab_size}")

Vocab size: 8963


In [9]:
# --- BƯỚC 3: DATA GENERATOR ---
def data_generator(sequences_digit, batch_size): # [cite: 23]
    while True:
        # Chọn ngẫu nhiên index cho batch
        batch_paths = np.random.choice(a=len(sequences_digit), size=batch_size) # [cite: 25]
        input_arr = []
        output_arr = []

        for i in batch_paths:
            input_arr.append(sequences_digit[i][:-1]) # Lấy tẩt cả trừ từ cuối [cite: 29]
            output_arr.append(sequences_digit[i][-1]) # Lấy từ cuối làm nhãn [cite: 30]

        input_arr = np.array(input_arr)
        # One-hot encoding cho output
        output_arr = keras.utils.to_categorical(output_arr, num_classes=vocab_size) # [cite: 32]
        output_arr = np.array(output_arr)

        yield (input_arr, output_arr) # [cite: 34]

In [10]:
# --- BƯỚC 4: XÂY DỰNG MÔ HÌNH ---
model = Sequential()
# Input length = 50 theo tài liệu [cite: 40]
model.add(Embedding(vocab_size, 150, input_length=50)) #
# Thêm 2 lớp LSTM như gợi ý
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax')) #

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model = load_model('model_weights.keras') #[cite: 69]

In [12]:
filepath_next = 'model_weights_phase2.keras'

checkpoint = ModelCheckpoint(
    filepath_next,
    monitor='accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)
callbacks_list = [checkpoint] #[cite: 52]

In [13]:
batch_size = 64

In [14]:
# --- BƯỚC 5: HUẤN LUYỆN THÊM 40 EPOCHS---
additional_epochs = 40

# Gọi lại generator (đảm bảo biến sequences_digit và batch_size đã có sẵn)
train_generator = data_generator(sequences_digit, batch_size) #[cite: 23]

history = model.fit(
    train_generator,
    steps_per_epoch=(len(sequences_digit)//batch_size),
    epochs=additional_epochs, # Sẽ chạy thêm 40 vòng nữa
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/40
1560/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3873 - loss: 3.1792
Epoch 1: accuracy improved from -inf to 0.40003, saving model to model_weights_phase2.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.3873 - loss: 3.1790
Epoch 2/40
1561/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4309 - loss: 2.9143
Epoch 2: accuracy improved from 0.40003 to 0.43924, saving model to model_weights_phase2.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.4310 - loss: 2.9142
Epoch 3/40
1560/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4689 - loss: 2.6943
Epoch 3: accuracy improved from 0.43924 to 0.47520, saving model to model_weights_phase2.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.4689 - loss: 2.6943
Epoch 4/40
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5026 - loss: 2.4941
Epoch 4: accuracy improved from 0.47520 to 0.51202, saving model to model_weights_phase2.keras
1562/1562 ━━━━━━━━━━━━━━━━━━━━

In [15]:
model.save('final_model.h5')

In [16]:
# --- BƯỚC 6: ỨNG DỤNG TẠO VĂN BẢN (INFERENCE) ---

# Tái định nghĩa tokenizer để dùng cho inference [cite: 61-65]
tokenizer = Tokenizer()
tokenizer.word_index = loaded_tokenizer.word_index
tokenizer.document_count = loaded_tokenizer.document_count
tokenizer.index_word = {v: k for k, v in loaded_tokenizer.word_index.items()}

In [17]:
# Hàm làm sạch văn bản [cite: 76]
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc) # [cite: 77]
    doc = doc.lower() # [cite: 78]
    tokens = doc.split() # [cite: 79]
    table = str.maketrans('', '', string.punctuation.replace("_", "")) # [cite: 80]
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

# Hàm tiền xử lý đầu vào [cite: 86]
def preprocess_input(doc):
    tokens = clean_document(doc)
    # Chuyển từ -> index
    sequence_of_indices = [tokenizer.word_index.get(word, 0) for word in tokens] # [cite: 89]
    # Padding về độ dài 50
    padded_sequence = pad_sequences([sequence_of_indices], maxlen=50, truncating='pre') # [cite: 91]
    return padded_sequence

# Hàm sinh văn bản [cite: 94]
def generate_text(text_input, n_words):
    tokens = preprocess_input(text_input)
    for _ in range(n_words): # [cite: 96]
        prediction = model.predict(tokens, verbose=0) # [cite: 97]
        next_digit = np.argmax(prediction, axis=-1) # [cite: 98]

        # Cập nhật chuỗi input: thêm từ mới, bỏ từ cũ nhất
        tokens = np.append(tokens, next_digit) # [cite: 99]
        tokens = np.delete(tokens, 0) # [cite: 100]
        tokens = np.reshape(tokens, (1, 50)) # [cite: 101]

    # Giải mã từ số về chữ
    tokens_flat = np.reshape(tokens, (50)) # [cite: 103]
    out_word = []
    for token in tokens_flat:
        if token != 0: # Bỏ qua padding [cite: 107]
            out_word.append(tokenizer.index_word.get(token, '')) # [cite: 108]

    return ' '.join(out_word) # [cite: 109]

In [19]:
# --- CHẠY THỬ ---
seq = 'việt nam' # [cite: 111]
generated_text = generate_text(text_input=seq, n_words=25) # [cite: 112]
print("Kết quả sinh văn bản:", generated_text)

Kết quả sinh văn bản: việt_nam nguyên_tắc nên chương_trình tiếp_tục sự_nghiệp khắc_phục giai_đoạn tỷ đảng toàn nhà tạo đầu_tư cho trên nhân_dân tp đà_nẵng width hoa_kỳ hoa_kỳ đất tại các vị_trí


In [20]:
# --- CHẠY THỬ ---
seq = 'Và đó là' # [cite: 111]
generated_text = generate_text(text_input=seq, n_words=25) # [cite: 112]
print("Kết quả sinh văn bản:", generated_text)

Kết quả sinh văn bản: và đó là bước tài_trợ toàn toàn dân_tộc hơn năm 2006 toàn toàn tỷ đồng nước đóng_góp của người dân và mục_tiêu kêu_gọi chương_trình đổi_mới cao đạo_đức xây_dựng
